In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import datetime

# --- 1. 데이터 임포트 ---
from services.tables.HR_Core.basic_info_table import emp_df
from services.tables.HR_Core.position_info_table import position_info_df
from services.tables.HR_Core.department_info_table import department_info_df
from services.tables.HR_Core.position_table import position_df
from services.tables.HR_Core.department_table import division_order, dept_level_map, parent_map_dept, dept_name_map
from services.helpers.utils import calculate_age, find_division_name_for_dept

def create_figure():
    """
    제안 3: 조직 세대교체 현황 분석 그래프를 생성합니다.
    """
    # --- 2. 데이터 준비 및 가공 ---
    current_emps_df = emp_df[emp_df['CURRENT_EMP_YN'] == 'Y'].copy()
    
    current_emps_df['AGE'] = current_emps_df['PERSONAL_ID'].apply(calculate_age)
    
    current_positions = position_info_df[position_info_df['GRADE_END_DATE'].isnull()][['EMP_ID', 'POSITION_ID']]
    current_depts = department_info_df[department_info_df['DEP_APP_END_DATE'].isnull()][['EMP_ID', 'DEP_ID']]
    
    analysis_df = pd.merge(current_emps_df, current_positions, on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, current_depts, on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, position_df[['POSITION_ID', 'POSITION_NAME']].drop_duplicates(), on='POSITION_ID', how='left')
    
    analysis_df['DIVISION_NAME'] = analysis_df['DEP_ID'].apply(lambda x: find_division_name_for_dept(x, dept_level_map, parent_map_dept, dept_name_map))
    
    analysis_df = analysis_df.dropna(subset=['POSITION_NAME', 'DIVISION_NAME', 'AGE'])
    
    position_order = ['Staff', 'Manager', 'Director', 'C-Level']
    analysis_df['POSITION_NAME'] = pd.Categorical(analysis_df['POSITION_NAME'], categories=position_order, ordered=True)
    analysis_df = analysis_df.sort_values('POSITION_NAME')
    
    y_min = analysis_df['AGE'].min()
    y_max = analysis_df['AGE'].max()
    fixed_y_range = [y_min - 5, y_max + 5]
    
    # --- 3. Plotly 인터랙티브 그래프 생성 ---
    fig = go.Figure()
    division_list = ['전체'] + division_order
    
    for i, div_name in enumerate(division_list):
        is_visible = (i == 0)
        df_filtered = analysis_df if div_name == '전체' else analysis_df[analysis_df['DIVISION_NAME'] == div_name]
        
        fig.add_trace(
            go.Violin(
                x=df_filtered['POSITION_NAME'], y=df_filtered['AGE'], name=div_name,
                box_visible=True, meanline_visible=True, visible=is_visible
            )
        )
        
    # --- 4. 드롭다운 메뉴 생성 ---
    buttons = []
    for i, div_name in enumerate(division_list):
        visibility_mask = [False] * len(division_list)
        visibility_mask[i] = True
        buttons.append(dict(label=div_name, method='update', args=[{'visible': visibility_mask}]))
        
    # --- 5. 레이아웃 업데이트 및 그래프 표시 ---
    fig.update_layout(
        updatemenus=[dict(
            active=0, buttons=buttons, direction="down",
            pad={"r": 10, "t": 10}, showactive=True,
            x=0.01, xanchor="left", y=1.1, yanchor="top"
        )],
        title_text='직위별 연령 분포 현황',
        xaxis_title='직위',
        yaxis_title='연령',
        font_size=14, height=700,
        legend_title_text='Division',
        annotations=[dict(text="조직 선택:", showarrow=False,
                          x=0, y=1.08, yref="paper", align="left")],
        yaxis_range=fixed_y_range
    )
    
    return fig

# 이 파일을 직접 실행할 경우 그래프를 생성하여 보여줍니다.
pio.renderers.default = 'vscode'
fig = create_figure()
fig.show()

In [2]:
# --- 2. 데이터 준비 및 가공 ---
current_emps_df = emp_df[emp_df['CURRENT_EMP_YN'] == 'Y'].copy()

# (calculate_age 함수가 공통 함수 블록에 정의되어 있다고 가정)
current_emps_df['AGE'] = current_emps_df['PERSONAL_ID'].apply(calculate_age)

current_positions = position_info_df[position_info_df['GRADE_END_DATE'].isnull()][['EMP_ID', 'POSITION_ID']]
current_depts = department_info_df[department_info_df['DEP_APP_END_DATE'].isnull()][['EMP_ID', 'DEP_ID']]

analysis_df = pd.merge(current_emps_df, current_positions, on='EMP_ID', how='left')
analysis_df = pd.merge(analysis_df, current_depts, on='EMP_ID', how='left')
analysis_df = pd.merge(analysis_df, position_df[['POSITION_ID', 'POSITION_NAME']].drop_duplicates(), on='POSITION_ID', how='left')

# (find_division_name_for_dept 함수가 공통 함수 블록에 정의되어 있다고 가정)
analysis_df['DIVISION_NAME'] = analysis_df['DEP_ID'].apply(lambda x: find_division_name_for_dept(x, dept_level_map, parent_map_dept, dept_name_map))

# --- 수정된 부분: inplace=True 제거 ---
analysis_df = analysis_df.dropna(subset=['POSITION_NAME', 'DIVISION_NAME', 'AGE'])
# --- 수정 완료 ---

position_order = ['Staff', 'Manager', 'Director', 'C-Level']
analysis_df['POSITION_NAME'] = pd.Categorical(analysis_df['POSITION_NAME'], categories=position_order, ordered=True)
analysis_df = analysis_df.sort_values('POSITION_NAME')

y_min = analysis_df['AGE'].min()
y_max = analysis_df['AGE'].max()
fixed_y_range = [y_min - 5, y_max + 5]

# --- 3. Plotly 인터랙티브 그래프 생성 ---
fig = go.Figure()
division_list = ['전체'] + division_order

for i, div_name in enumerate(division_list):
    is_visible = (i == 0)
    df_filtered = analysis_df if div_name == '전체' else analysis_df[analysis_df['DIVISION_NAME'] == div_name]

    fig.add_trace(
        go.Violin(
            x=df_filtered['POSITION_NAME'], y=df_filtered['AGE'], name=div_name,
            box_visible=True, meanline_visible=True, visible=is_visible
        )
    )

# --- 4. 드롭다운 메뉴 생성 ---
buttons = []
for i, div_name in enumerate(division_list):
    visibility_mask = [False] * len(division_list)
    visibility_mask[i] = True
    buttons.append(dict(label=div_name, method='update', args=[{'visible': visibility_mask}]))

# --- 5. 레이아웃 업데이트 및 그래프 표시 ---
fig.update_layout(
    updatemenus=[dict(
        active=0, buttons=buttons, direction="down",
        pad={"r": 10, "t": 10}, showactive=True,
        x=0.01, xanchor="left", y=1.1, yanchor="top"
    )],
    title_text='직위별 연령 분포 현황',
    xaxis_title='직위',
    yaxis_title='연령',
    font_size=14, height=700,
    legend_title_text='Division',
    annotations=[dict(text="조직 선택:", showarrow=False,
                      x=0, y=1.08, yref="paper", align="left")],
    yaxis_range=fixed_y_range
)
fig.show()

In [5]:
analysis_df

,EMP_ID,NAME,ENG_NAME,NICKNAME,PERSONAL_ID,GENDER,EMAIL,PHONE_NUM,IN_DATE,GROUP_IN_DATE,...,ADDRESS,CURRENT_EMP_YN,OUT_DATE,DURATION,PROB_YN,AGE,POSITION_ID,DEP_ID,POSITION_NAME,DIVISION_NAME
239,E00475,김현우,Kenneth Adams,Christina,980424-1134913,M,mscott@example.com,02-1693-4292,2022-10-12,2021-09-03,...,인천광역시 광진구 언주길 701-51,Y,NaT,1042,N,27,POS001,DEP013,Staff,Operating Division
149,E00293,이성민,Melinda Mcdaniel,Jennifer,980726-1693169,M,hjackson@example.org,033-451-9515,2024-05-28,2024-05-28,...,서울특별시 동작구 역삼가 615,Y,NaT,448,N,27,POS001,DEP035,Staff,Operating Division
358,E00743,이영환,Todd Wright,Ryan,901111-1475394,M,crystallandry@example.org,063-818-7109,2020-08-24,2020-08-24,...,울산광역시 도봉구 삼성가 658-71 (성수이마을),Y,NaT,1821,N,35,POS001,DEP031,Staff,Operating Division
151,E00296,전영철,Jennifer Clark,Jeffrey,950427-2551640,F,haydenjane@example.com,070-0650-1917,2023-02-15,2023-02-15,...,경기도 평택시 테헤란3로 852-54,Y,NaT,916,N,30,POS001,DEP014,Staff,Operating Division
153,E00299,최광수,Mrs. Amber Davis,Kyle,000322-4480292,F,susan83@example.com,055-521-5681,2018-02-14,2018-02-14,...,경기도 논산시 서초중앙042길 지하81,Y,NaT,2743,N,25,POS001,DEP016,Staff,Planning Division
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,E00749,한영순,Shirley Ray,Pamela,950504-1275722,M,keithrich@example.net,055-792-4157,2011-11-07,2011-11-07,...,경상북도 고양시 삼성거리 421 (명자이장동),Y,NaT,5034,N,30,POS003,DEP030,Director,Sales Division
360,E00746,성정호,Sarah Wilson,Jared,901106-2175073,F,jeffreygarcia@example.org,017-252-8613,2012-12-28,2012-12-28,...,대전광역시 서구 오금52길 513-52 (은경이박동),Y,NaT,4617,N,35,POS003,DEP012,Director,Sales Division
158,E00314,이상철,Kathleen Mooney,Robert,820327-2910260,F,carolyngarza@example.org,011-040-9074,2012-08-02,2012-08-02,...,경기도 양양군 삼성로 98-54 (예준권이마을),Y,NaT,4765,N,43,POS003,DEP021,Director,Development Division
190,E00389,김명숙,Craig Wise,Brittany,820401-2829822,F,williamlane@example.net,033-231-7955,2016-10-28,2016-10-28,...,강원도 화천군 서초중앙81가 390-54,Y,NaT,3217,N,43,POS004,DEP022,C-Level,Development Division


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import datetime

# --- 1. 데이터 임포트 ---
from services.tables.HR_Core.basic_info_table import emp_df
from services.tables.HR_Core.position_info_table import position_info_df
from services.tables.HR_Core.department_info_table import department_info_df
from services.tables.HR_Core.position_table import position_df
from services.tables.HR_Core.department_table import division_order, office_order, department_df
from services.helpers.utils import calculate_age

def create_figure():
    """
    제안 3: 조직 세대교체 현황 분석 그래프를 생성합니다. (Office 드릴다운 추가)
    """
    # --- 2. 데이터 준비 및 가공 ---
    current_emps_df = emp_df[emp_df['CURRENT_EMP_YN'] == 'Y'].copy()
    current_emps_df['AGE'] = current_emps_df['PERSONAL_ID'].apply(calculate_age)

    current_positions = position_info_df[position_info_df['GRADE_END_DATE'].isnull()][['EMP_ID', 'POSITION_ID']]
    current_depts = department_info_df[department_info_df['DEP_APP_END_DATE'].isnull()][['EMP_ID', 'DEP_ID']]
    
    analysis_df = pd.merge(current_emps_df, current_positions, on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, current_depts, on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, position_df[['POSITION_ID', 'POSITION_NAME']].drop_duplicates(), on='POSITION_ID', how='left')

    div_map = department_df[department_df['DEP_LEVEL'] == 2][['DEP_ID', 'DEP_NAME']].rename(columns={'DEP_NAME': 'DIVISION_NAME'})
    office_map = pd.merge(department_df[department_df['DEP_LEVEL'] == 3], div_map, left_on='UP_DEP_ID', right_on='DEP_ID', suffixes=('', '_div'))[['DEP_ID', 'DIVISION_NAME']]
    team_map = pd.merge(department_df[department_df['DEP_LEVEL'] == 4], office_map, left_on='UP_DEP_ID', right_on='DEP_ID', suffixes=('', '_office'))[['DEP_ID', 'DIVISION_NAME']]
    dept_to_div_map = pd.concat([
        div_map.rename(columns={'DEP_ID': 'DEP_ID_MAP'}),
        office_map.rename(columns={'DEP_ID': 'DEP_ID_MAP'}),
        team_map.rename(columns={'DEP_ID': 'DEP_ID_MAP'})
    ]).rename(columns={'DEP_ID_MAP': 'DEP_ID'})
    analysis_df = pd.merge(analysis_df, dept_to_div_map[['DEP_ID', 'DIVISION_NAME']], on='DEP_ID', how='left')
    
    # Office 이름을 붙이기 위해 department_df와 merge
    analysis_df = pd.merge(analysis_df, department_df[['DEP_ID', 'DEP_NAME']], on='DEP_ID', how='left')
    analysis_df['OFFICE_NAME'] = np.where(analysis_df['DEP_NAME'].str.contains('Office'), analysis_df['DEP_NAME'], analysis_df['DIVISION_NAME'] + ' (직속)')
    
    analysis_df = analysis_df.dropna(subset=['POSITION_NAME', 'DIVISION_NAME', 'AGE', 'OFFICE_NAME'])
    
    position_order = ['Staff', 'Manager', 'Director', 'C-Level']
    analysis_df['POSITION_NAME'] = pd.Categorical(analysis_df['POSITION_NAME'], categories=position_order, ordered=True)
    analysis_df['DIVISION_NAME'] = pd.Categorical(analysis_df['DIVISION_NAME'], categories=division_order, ordered=True)
    analysis_df['OFFICE_NAME'] = pd.Categorical(analysis_df['OFFICE_NAME'], categories=office_order, ordered=True)
    analysis_df = analysis_df.sort_values(['DIVISION_NAME', 'OFFICE_NAME', 'POSITION_NAME'])

    y_min, y_max = analysis_df['AGE'].min(), analysis_df['AGE'].max()
    fixed_y_range = [y_min - 5, y_max + 5]

    # --- 3. Plotly 인터랙티브 그래프 생성 ---
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    
    # 1. '전체' 뷰 트레이스 추가 (Division 기준)
    for i, div_name in enumerate(division_order):
        df_filtered = analysis_df[analysis_df['DIVISION_NAME'] == div_name]
        fig.add_trace(go.Box(x=df_filtered['POSITION_NAME'], y=df_filtered['AGE'], name=div_name, marker_color=colors[i]))

    # 2. '상세' 뷰 트레이스 추가 (Office 기준, 초기는 숨김)
    office_traces_map = {}
    trace_idx_counter = len(fig.data)
    for i, div_name in enumerate(division_order):
        office_div_df = analysis_df[analysis_df['DIVISION_NAME'] == div_name]
        offices_in_div = [o for o in office_order if o in office_div_df['OFFICE_NAME'].unique()]
        office_traces_map[div_name] = []
        for j, office_name in enumerate(offices_in_div):
            df_filtered = office_div_df[office_div_df['OFFICE_NAME'] == office_name]
            fig.add_trace(go.Box(
                x=df_filtered['POSITION_NAME'], y=df_filtered['AGE'], name=office_name, 
                visible=False, marker_color=colors[j % len(colors)]
            ))
            office_traces_map[div_name].append(trace_idx_counter)
            trace_idx_counter += 1

    # --- 4. 드롭다운 메뉴 및 레이아웃 업데이트 ---
    buttons = []
    buttons.append(dict(label='전체', method='update', 
                        args=[{'visible': [True]*len(division_order) + [False]*(len(fig.data)-len(division_order))},
                              {'title': '전체 조직의 직위별 연령 분포', 'legend_title_text': 'Division'}]))

    for div_name in division_order:
        visibility_mask = [False] * len(fig.data)
        for trace_idx in office_traces_map.get(div_name, []):
            visibility_mask[trace_idx] = True
        buttons.append(dict(label=f'{div_name}', method='update',
                            args=[{'visible': visibility_mask},
                                  {'title': f'{div_name} 내 직위별 연령 분포', 'legend_title_text': 'Office'}]))

    fig.update_layout(
        updatemenus=[dict(
            active=0, buttons=buttons, direction="down",
            pad={"r": 10, "t": 10}, showactive=True,
            x=0.01, xanchor="left", y=1.1, yanchor="top"
        )],
        title_text='직위별 연령 분포 현황',
        xaxis_title='직위', yaxis_title='연령',
        font_size=14, height=700,
        boxmode='group',
        legend_title_text='Division',
        annotations=[dict(text="조직 선택:", showarrow=False, x=0, y=1.08, yref="paper", align="left")],
        yaxis_range=fixed_y_range
    )
    
    return fig

# 이 파일을 직접 실행할 경우 그래프를 생성하여 보여줍니다.
if __name__ == '__main__':
    pio.renderers.default = 'vscode'
    fig = create_figure()
    fig.show()